In [1]:
import sys
import time
import os

from clangParser.clangParser import parse_project, parsing
from clangParser.CUnit import CUnit
from clangParser.Cursor import Cursor

from clangParser.CursorOMS import CursorOMS

from clang.cindex import Cursor as clangCursor


start_time = time.time()
project_unit_list:[] = []
project_unit_list[:0] = parse_project(r"D:\dev\AutoPlanning\trunk\AP_trunk_pure\Appframe")
project_unit_list[:0] = parse_project(r"D:\dev\AutoPlanning\trunk\AP_trunk_pure\AppUICore")
project_unit_list[:0] = parse_project(r"D:\dev\AutoPlanning\trunk\AP_trunk_pure\mod_APImplantSimulation")
project_unit_list[:0] = parse_project(r"D:\dev\AutoPlanning\trunk\AP_trunk_pure\mod_APReady")
project_unit_list[:0] = parse_project(r"D:\dev\AutoPlanning\trunk\AP_trunk_pure\mod_APScanEdit")
project_unit_list[:0] = parse_project(r"D:\dev\AutoPlanning\trunk\AP_trunk_pure\mod_APSurgicalGuide")
project_unit_list[:0] = parse_project(r"D:\dev\AutoPlanning\trunk\AP_trunk_pure\mod_APWorkSpace")
end_time = time.time()
elapsed_time = end_time - start_time

index = 0
for node in project_unit_list:
    print(f"[{index}]{node.cursor.spelling}")
    index += 1
    
print("project parse time :", elapsed_time, "seconds")

parse D:\dev\AutoPlanning\trunk\AP_trunk_pure\Appframe\AbutmentData.cpp
parse D:\dev\AutoPlanning\trunk\AP_trunk_pure\Appframe\AbutmentData.h
parse D:\dev\AutoPlanning\trunk\AP_trunk_pure\Appframe\AbutmentDoc.cpp
parse D:\dev\AutoPlanning\trunk\AP_trunk_pure\Appframe\AbutmentDoc.h
parse D:\dev\AutoPlanning\trunk\AP_trunk_pure\Appframe\AppFrame.cpp
parse D:\dev\AutoPlanning\trunk\AP_trunk_pure\Appframe\Base_Data.cpp
parse D:\dev\AutoPlanning\trunk\AP_trunk_pure\Appframe\Base_Data.h
parse D:\dev\AutoPlanning\trunk\AP_trunk_pure\Appframe\BiteAlignmentData.cpp
parse D:\dev\AutoPlanning\trunk\AP_trunk_pure\Appframe\BiteAlignmentData.h
parse D:\dev\AutoPlanning\trunk\AP_trunk_pure\Appframe\CrownDesignData.cpp
parse D:\dev\AutoPlanning\trunk\AP_trunk_pure\Appframe\CrownDesignData.h
parse D:\dev\AutoPlanning\trunk\AP_trunk_pure\Appframe\CrownDoc.cpp
parse D:\dev\AutoPlanning\trunk\AP_trunk_pure\Appframe\CrownDoc.h
parse D:\dev\AutoPlanning\trunk\AP_trunk_pure\Appframe\DataMgrDoc.cpp
parse D:\d

In [2]:
#cindex.unit to CUnit
start_time = time.time()
my_unit_list =[]
def_node_set = set()
for clang_unit in project_unit_list:
    my_unit_list.append(CUnit(clang_unit))
end_time = time.time()
unit_gen_time = end_time - start_time
print("CUnit gen time :", unit_gen_time, "seconds")

CUnit gen time : 48.75817561149597 seconds


In [3]:
import oms.Mapper as Mapper

start_time = time.time()

#CUnit to nodes
all_node_list = []
for unit in my_unit_list:
    all_node_list[:0] = unit.this_file_nodes
    
#nodes to OMS
all_info_oms, clang_src_map  = Mapper.parsing(all_node_list)
end_time = time.time()
mapping_time = end_time - start_time
print("mapping time :", mapping_time, "seconds")



Mapping (100.00%) 8223/8223
Mapping done
Update Call (100.00%) 6924/6924
Update done
mapping time : 2622.556507587433 seconds


In [9]:
from collections import defaultdict
line_size_map = defaultdict(list)

for fun_info in all_info_oms.functionInfos.values():
    line_num = fun_info.code.count('\n')
    line_size_map[line_num].append(fun_info)    

In [10]:
for line_num, data_list in sorted(line_size_map.items()):
    end = "\n"
    if len(data_list) == 1:
        end = data_list[0].src_name +"\n"
    print(f"{line_num} line : {len(data_list)} methods " , end = end)

0 line : 1463 methods 
1 line : 128 methods 
2 line : 274 methods 
3 line : 840 methods 
4 line : 293 methods 
5 line : 411 methods 
6 line : 370 methods 
7 line : 264 methods 
8 line : 221 methods 
9 line : 193 methods 
10 line : 184 methods 
11 line : 173 methods 
12 line : 154 methods 
13 line : 208 methods 
14 line : 159 methods 
15 line : 142 methods 
16 line : 107 methods 
17 line : 143 methods 
18 line : 124 methods 
19 line : 127 methods 
20 line : 124 methods 
21 line : 102 methods 
22 line : 104 methods 
23 line : 110 methods 
24 line : 91 methods 
25 line : 69 methods 
26 line : 113 methods 
27 line : 84 methods 
28 line : 77 methods 
29 line : 65 methods 
30 line : 85 methods 
31 line : 88 methods 
32 line : 59 methods 
33 line : 52 methods 
34 line : 37 methods 
35 line : 40 methods 
36 line : 44 methods 
37 line : 48 methods 
38 line : 41 methods 
39 line : 39 methods 
40 line : 40 methods 
41 line : 39 methods 
42 line : 47 methods 
43 line : 45 methods 
44 line : 42 met

In [11]:
# 특정 라인수 출력
# for fun_info in line_size_map[3]:
#     print("===")
#     print(fun_info.code)

method_list = []
all_count = 0
for line_num, data_list in line_size_map.items():
    all_count += len(data_list)
    if 2 < line_num:
        method_list += data_list

print(all_count, " -> ", len(method_list))
    

8871  ->  7006


In [12]:
# !wget https://raw.githubusercontent.com/microsoft/CodeBERT/master/UniXcoder/unixcoder.py

In [13]:
import torch
from transformers import RobertaTokenizer, RobertaConfig, RobertaModel

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = RobertaTokenizer.from_pretrained("microsoft/longcoder-base")
model = RobertaModel.from_pretrained("microsoft/longcoder-base")
model.to(device)

C:\dev\Anaconda\envs\T1\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
You are using a model of type longformer to instantiate a model of type roberta. This is not supported for all configurations of models and can yield errors.


RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(51416, 768, padding_idx=1)
    (position_embeddings): Embedding(4098, 768, padding_idx=1)
    (token_type_embeddings): Embedding(10, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0-11): 12 x RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (drop

In [14]:
all_code_list = [fun_info.code for fun_info in method_list]
all_tokens_list = [tokenizer.tokenize(code_snippet) for code_snippet in code_list]
all_tokens_ids_list = [torch.tensor(tokenizer.convert_tokens_to_ids(tokens)) for tokens in tokens_list]

In [63]:
BERT_MAX_SIZE = 4098
target_method_list = []
code_list = []
tokens_list = []
tokens_ids_list = []
for idx, idx_list in enumerate(all_tokens_ids_list):
    if len(idx_list) < 4098:
        code_list.append(all_code_list[idx])
        tokens_list.append(all_tokens_list[idx])
        tokens_ids_list.append(idx_list)
        target_method_list.append(method_list[idx])

print(len(all_tokens_ids_list), " -> ", len(tokens_ids_list))

7006  ->  6988


In [36]:
token_size_map = defaultdict(list)
for idx, tokens in enumerate(tokens_list):
    token_size_map[len(tokens)].append((tokens, idx))

for token_num, item in sorted(token_size_map.items()):
    end = "\n"
    if len(item) == 1:
        idx = item[0][1]
        end = f"{method_list[idx].src_name} \n"
    print(f"{token_num} tokens : {len(item)} codes", end=end)
    

12 tokens : 1 codesCToolbarSetting.OnNext() 
13 tokens : 5 codes
14 tokens : 2 codes
15 tokens : 2 codes
16 tokens : 9 codes
17 tokens : 8 codes
18 tokens : 24 codes
19 tokens : 16 codes
20 tokens : 18 codes
21 tokens : 13 codes
22 tokens : 24 codes
23 tokens : 25 codes
24 tokens : 44 codes
25 tokens : 44 codes
26 tokens : 44 codes
27 tokens : 36 codes
28 tokens : 37 codes
29 tokens : 47 codes
30 tokens : 56 codes
31 tokens : 37 codes
32 tokens : 55 codes
33 tokens : 49 codes
34 tokens : 51 codes
35 tokens : 51 codes
36 tokens : 46 codes
37 tokens : 37 codes
38 tokens : 42 codes
39 tokens : 46 codes
40 tokens : 49 codes
41 tokens : 44 codes
42 tokens : 43 codes
43 tokens : 31 codes
44 tokens : 33 codes
45 tokens : 32 codes
46 tokens : 34 codes
47 tokens : 39 codes
48 tokens : 38 codes
49 tokens : 43 codes
50 tokens : 37 codes
51 tokens : 34 codes
52 tokens : 23 codes
53 tokens : 33 codes
54 tokens : 34 codes
55 tokens : 29 codes
56 tokens : 37 codes
57 tokens : 41 codes
58 tokens : 48 

In [16]:
device = 'cuda'
# device = next(tokenizer.parameters()).device
# print(device)

In [17]:
print(padded_tokens_ids.shape)
print(model)

NameError: name 'padded_tokens_ids' is not defined

In [19]:
idx = 10
print(method_list[idx].src_name)
print("="*100)
print(method_list[idx].code)
print("="*100)
print(token_list[idx])

CNamingBarData.GetReturnLoopNum(int, bool)
int CNamingBarData::GetReturnLoopNum(int nNamingBarIndex, bool bUpper)
{
	int nIndex;
	if (bUpper)
	{
		nIndex = nNamingBarIndex - SURGICALGUIDE_UPPER_NAMINGBAR_INDEX;
	}
	else
	{
		nIndex = nNamingBarIndex - SURGICALGUIDE_LOWER_NAMINGBAR_INDEX;
	}

	int nLoopNum = nIndex / 100;

	return nLoopNum;
}


NameError: name 'token_list' is not defined

In [32]:

# del cuda_padded_tokens_ids
# del cuda_attention_mask
# del cuda_context_embeddings
print(padded_tokens_ids.shape)
print(attention_mask.shape)
print(model)
torch.cuda.empty_cache()

torch.Size([7006, 20293])
torch.Size([7006, 20293])
RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(51416, 768, padding_idx=1)
    (position_embeddings): Embedding(4098, 768, padding_idx=1)
    (token_type_embeddings): Embedding(10, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0-11): 12 x RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), e

In [205]:
print(len(tokens_ids_list))
def get_batch_token_ids(index, batch_size = 16):
    #원본 배치 기준 계
    num_batches = len(tokens_ids_list) // batch_size + (len(tokens_ids_list) % batch_size > 0)

    # 해당 인덱스가 포함된 배치의 번호 계산
    batch_index = index // batch_size
    print("batch_index ", batch_index)
    
    # 배치의 시작과 끝 인덱스
    start_idx = batch_index * batch_size
    end_idx = min((batch_index + 1) * batch_size, len(tokens_ids_list))
    print("start ", start_idx)
    
    # 배치 내의 토큰 ID들 추출
    batch_tokens_ids = tokens_ids_list[start_idx:end_idx]
    
    # 패딩을 적용하여 동일한 길이로 맞춤
    padded_tokens_ids = pad_sequence(batch_tokens_ids, batch_first=True, padding_value=0)

    # 어텐션 마스크 생성
    attention_mask = torch.zeros_like(padded_tokens_ids).long()
    for j, tokens_ids in enumerate(batch_tokens_ids):
        attention_mask[j, :len(tokens_ids)] = 1
    
     # 결과 반환
    return padded_tokens_ids, attention_mask, batch_tokens_ids

target_pad_tokens, target_attention_mask, target_batch_tokens_ids = get_batch_token_ids(index = 0)

for tokens in target_batch_tokens_ids:
    print(len(tokens))

6988
batch_index  0
start  0
109
125
183
226
221
231
94
128
189
177
122
133
117
194
119
166


In [46]:
#실질적인 수행 코드 상당히 오래걸림
batch_size = 16  # 예시로 작은 배치 크기를 설정
num_batches = len(tokens_ids_list) // batch_size + (len(tokens_ids_list) % batch_size > 0)

embeded_tokens_list = []

for i in range(num_batches):
    batch_tokens_ids = tokens_ids_list[i * batch_size: (i + 1) * batch_size]
    padded_tokens_ids = pad_sequence(batch_tokens_ids, batch_first=True, padding_value=0)

    attention_mask = torch.zeros_like(padded_tokens_ids).long()
    for j, tokens_ids in enumerate(batch_tokens_ids):
        attention_mask[j, :len(tokens_ids)] = 1

    # GPU로 이동
    cuda_padded_tokens_ids = padded_tokens_ids.to('cuda')
    cuda_attention_mask = attention_mask.to('cuda')

    # 모델 추론
    with torch.no_grad():
        cuda_context_embeddings = model(cuda_padded_tokens_ids, attention_mask=cuda_attention_mask)[0]

    embeded_tokens_list.append(cuda_context_embeddings.to('cpu'))

    del cuda_padded_tokens_ids
    del cuda_attention_mask
    del cuda_context_embeddings
    torch.cuda.empty_cache()

# 최종 결과를 하나의 텐서로 병합
embeded_tokens = torch.cat(embeded_tokens_list, dim=0)


RuntimeError: Sizes of tensors must match except in dimension 0. Expected size 231 but got size 372 for tensor number 1 in the list.

In [285]:
print(len(target_method_list))
print(len(tokens_ids_list))
print(len(embeded_tokens))

from oms.dataset.function_info import FunctionInfo

#일단 클래스로 관리하는게 나을것 같긴하다. 늦었지만...
class EmbededMethodInfo:
    def __init__(self, fun_info, str_tokens, nId_tokens, vector_tokens):
        self.fun_info: FunctionInfo = fun_info
        self.str_tokens: [str] = str_tokens
        self.nId_tokens : [int] = nId_tokens
        self.vec_tokens : [torch.tensor] = vector_tokens

t_start = time.time()
DataSet = []
for i in range(len(target_method_list)):
    
    fun_info = target_method_list[i]
    nId_tokens = tokens_ids_list[i]
    str_tokens = tokenizer.convert_ids_to_tokens(nId_tokens)
    vector_tokens = []
    for j in range(len(nId_tokens)): 
        vector_tokens.append(embeded_tokens[j])    
    data = EmbededMethodInfo(fun_info, str_tokens, nId_tokens, vector_tokens)
    DataSet.append(data)
t_end = time.time()

print(len(DataSet))

        

6988
6988
6988
6988


In [290]:
d = DataSet[20]
print(d.fun_info.code)
print("=="*100)
print(d.str_tokens)
print(len(d.str_tokens),"==",len(d.nId_tokens))
print(len(d.vec_tokens))
for v in d.vec_tokens:
    print(v.shape)

void CUIPane::OnMouseHover(UINT nFlags, CPoint point)
{
	if (_bMouseHover == false)
	{
		_bMouseHover = true;
		GetParent()->PostMessage(WM_USER_PANE_HOVER, (WPARAM)_bMouseHover);
	}

	if (m_pActuator)
	{
		if (m_pActuator->OnMouseHover(nFlags, point))
			m_pActuator->UpdateActuator();
	}

	__super::OnMouseHover(nFlags, point);
}
['void', 'ĠCU', 'IP', 'ane', '::', 'On', 'Mouse', 'Hover', '(', 'UINT', 'Ġn', 'Flags', ',', 'ĠC', 'Point', 'Ġpoint', ')', 'Ċ', '{', 'Ċ', 'ĉ', 'if', 'Ġ(_', 'b', 'Mouse', 'Hover', 'Ġ==', 'Ġfalse', ')', 'Ċ', 'ĉ', '{', 'Ċ', 'ĉ', 'ĉ', '_', 'b', 'Mouse', 'Hover', 'Ġ=', 'Ġtrue', ';', 'Ċ', 'ĉ', 'ĉ', 'GetParent', '()->', 'Post', 'Message', '(', 'WM', '_', 'USER', '_', 'PAN', 'E', '_', 'HO', 'VER', ',', 'Ġ(', 'WPARAM', ')_', 'b', 'Mouse', 'Hover', ');', 'Ċ', 'ĉ', '}', 'Ċ', 'Ċ', 'ĉ', 'if', 'Ġ(', 'm', '_', 'p', 'Act', 'uator', ')', 'Ċ', 'ĉ', '{', 'Ċ', 'ĉ', 'ĉ', 'if', 'Ġ(', 'm', '_', 'p', 'Act', 'uator', '->', 'On', 'Mouse', 'Hover', '(', 'n', 'Flags', ',', 'Ġpoint', '))',

In [184]:
# shape_set = set()
# for tokens_list in embeded_tokens_list:
#     shape_set.add(tokens_list.shape)
# shape_set


# len_set = set()
# for tokens_list in embeded_tokens_list:
#     for tokens in tokens_list:
#         len_set.add(len(tokens))
#     # len_set.add(len(tokens_list))
# len_set

embeded_tokens = []
for batch_embeded_tokens in embeded_tokens_list:
    for tokens_list in batch_embeded_tokens:
        embeded_tokens.append(tokens_list)

print(len(target_method_list))
print(len(tokens_ids_list))
print(len(embeded_tokens))

dif_len=[]
for idx, vector_tokens in enumerate(embeded_tokens):
    if len(tokens_ids_list[idx]) == len(vector_tokens):
        continue
    else:
        dif_len.append(idx)
        if len(tokens_ids_list[idx]) < len(vector_tokens):
            print(idx," 토큰수가 증가 ", len(vector_tokens) - len(tokens_ids_list[idx]))
print(len(dif_len))

6988
6988
6988
0  토큰수가 증가  122
1  토큰수가 증가  106
2  토큰수가 증가  48
3  토큰수가 증가  5
4  토큰수가 증가  10
6  토큰수가 증가  137
7  토큰수가 증가  103
8  토큰수가 증가  42
9  토큰수가 증가  54
10  토큰수가 증가  109
11  토큰수가 증가  98
12  토큰수가 증가  114
13  토큰수가 증가  37
14  토큰수가 증가  112
15  토큰수가 증가  65
16  토큰수가 증가  193
17  토큰수가 증가  225
18  토큰수가 증가  208
19  토큰수가 증가  205
20  토큰수가 증가  235
21  토큰수가 증가  254
22  토큰수가 증가  214
23  토큰수가 증가  237
24  토큰수가 증가  256
25  토큰수가 증가  273
26  토큰수가 증가  247
28  토큰수가 증가  150
29  토큰수가 증가  148
30  토큰수가 증가  173
31  토큰수가 증가  253
32  토큰수가 증가  177
33  토큰수가 증가  179
34  토큰수가 증가  97
35  토큰수가 증가  167
36  토큰수가 증가  99
37  토큰수가 증가  193
38  토큰수가 증가  76
39  토큰수가 증가  97
40  토큰수가 증가  64
41  토큰수가 증가  67
42  토큰수가 증가  107
43  토큰수가 증가  175
44  토큰수가 증가  173
46  토큰수가 증가  48
47  토큰수가 증가  171
48  토큰수가 증가  117
49  토큰수가 증가  54
51  토큰수가 증가  1
52  토큰수가 증가  52
53  토큰수가 증가  53
54  토큰수가 증가  124
55  토큰수가 증가  124
56  토큰수가 증가  124
57  토큰수가 증가  1
58  토큰수가 증가  71
59  토큰수가 증가  53
60  토큰수가 증가  73
61  토큰수가 증가  132
62  토큰수가 증가  79
63  토큰수가 증가  151
6

In [ ]:
tokenizer.

In [206]:
idx = 0
print(target_method_list[idx].code)
print(len(tokens_ids_list[idx]))
print(len(embeded_tokens[idx]))

target_method = target_method_list[idx]

bool CXMLDocSinusLift::XmlLoad()
{

	bool bResult = false;
	_docResult.Clear();

	CString strPath;
	_getSinusLiftXmlPath(strPath);

	tinyxml2::XMLError error = CTinyXMLWrapper::LoadFile(&_docResult, strPath);

	if (error == tinyxml2::XMLError::XML_SUCCESS)
		bResult = true;

	return bResult;
}
109
231


In [129]:
del cuda_result
del cuda_tokens_ids
del result

NameError: name 'cuda_result' is not defined

In [207]:
tokens = tokenizer.tokenize(target_method.code)
print(len(tokens))
print(len(tokens_ids_list[idx]))

# print(tokens ==  tokenizer.convert_ids_to_tokens(tokens_ids_list[idx]))
print(tokens_ids_list[idx] ==  tokenizer.convert_tokens_to_ids(tokens))
ip_data = tokens_ids_list[idx][None,:]
print("ip : ", ip_data.shape)

with torch.no_grad():
    cuda_tokens_ids = ip_data.to('cuda')
    cuda_result = model(cuda_tokens_ids)[0]
    result = cuda_result.to('cpu')
    del cuda_tokens_ids
    del cuda_result

result.shape

109
109
False
ip :  torch.Size([1, 109])


torch.Size([1, 109, 768])

In [178]:
result584 = embeded_tokens[200]
result584.shape

torch.Size([584, 768])

In [179]:
result110 = embeded_tokens[532]
result110.shape

torch.Size([110, 768])

In [210]:
result231 = embeded_tokens[0]
result231.shape

torch.Size([231, 768])

In [234]:
tokens_ids_list[0].shape

torch.Size([109])

In [260]:
t_start=time.time()
token_id_vector_map = defaultdict(list)
for text_id, ids in enumerate(tokens_ids_list):
    token_vectors = embeded_tokens[text_id]
    str_tokens = tokenizer.convert_ids_to_tokens(ids)
    for idx in range(len(ids)):
        str_token = str_tokens[idx]
        token_id_vector_map[str_token].append((token_vectors[idx], text_id))
t_end = time.time()
print(t_end-t_start)

21.48195791244507


In [256]:
len(size_map[2])

795

In [267]:
print(len(token_id_vector_map))
size_map = defaultdict(list)
for key, vectors in token_id_vector_map.items():
    size_map[len(vectors)].append(key)
    
print(len(size_map))

for size, token_ids in sorted(size_map.items()):
    end = f"{len(token_ids)} tokens "
    if len(token_ids) < 10:
        tks = [_id for _id in token_ids]
        for t in tks:
            end += t+" "
    end += "\n"
    print(f"{size} : ", end = end)
    
    

7896
940
1 : 1115 tokens 
2 : 795 tokens 
3 : 486 tokens 
4 : 388 tokens 
5 : 261 tokens 
6 : 265 tokens 
7 : 196 tokens 
8 : 213 tokens 
9 : 151 tokens 
10 : 145 tokens 
11 : 102 tokens 
12 : 124 tokens 
13 : 89 tokens 
14 : 102 tokens 
15 : 86 tokens 
16 : 107 tokens 
17 : 59 tokens 
18 : 69 tokens 
19 : 72 tokens 
20 : 47 tokens 
21 : 52 tokens 
22 : 54 tokens 
23 : 57 tokens 
24 : 53 tokens 
25 : 61 tokens 
26 : 45 tokens 
27 : 46 tokens 
28 : 43 tokens 
29 : 33 tokens 
30 : 35 tokens 
31 : 32 tokens 
32 : 37 tokens 
33 : 35 tokens 
34 : 27 tokens 
35 : 27 tokens 
36 : 21 tokens 
37 : 21 tokens 
38 : 26 tokens 
39 : 25 tokens 
40 : 24 tokens 
41 : 27 tokens 
42 : 16 tokens 
43 : 16 tokens 
44 : 18 tokens 
45 : 17 tokens 
46 : 22 tokens 
47 : 16 tokens 
48 : 25 tokens 
49 : 15 tokens 
50 : 20 tokens 
51 : 25 tokens 
52 : 19 tokens 
53 : 17 tokens 
54 : 16 tokens 
55 : 14 tokens 
56 : 17 tokens 
57 : 13 tokens 
58 : 14 tokens 
59 : 15 tokens 
60 : 13 tokens 
61 : 18 tokens 
62 : 17 t

In [280]:
path_vectors=token_id_vector_map['path']
len(path_vectors)

shape_set = set()

vectors = [ infos[0] for infos in path_vectors]

base_data_idx = 243
base_i = 0
for i in range(len(vectors)):
    infos = path_vectors[i]
    cur_id = infos[1]
    
    cosine_similarity = F.cosine_similarity(vectors[i], vectors[base_i], dim=0)
    src_name = method_list[cur_id].src_name
    print(f"{cur_id}-{base_data_idx} {cosine_similarity:.2f} : {cur_id} {src_name}")
    if 10 < abs(base_data_idx-cur_id):
        base_data_idx = cur_id
        base_i = i
    

print(shape_set)

243-243 1.00 : 243 CSurfaceModelDoc.Load(LPCWSTR)
243-243 0.98 : 243 CSurfaceModelDoc.Load(LPCWSTR)
244-243 0.99 : 244 CSurfaceModelDoc.Save(LPCWSTR)
244-243 0.96 : 244 CSurfaceModelDoc.Save(LPCWSTR)
310-243 0.62 : 310 CCrownCtrlPtsDocs.Write(const CString&, int)
311-310 0.93 : 311 CCrownCtrlPtsDocs.Read(const CString&)
669-310 0.65 : 669 CTransferFuncManager.Load(CString)
669-669 1.00 : 669 CTransferFuncManager.Load(CString)
670-669 1.00 : 670 CTransferFuncManager.Save(CString)
670-669 0.99 : 670 CTransferFuncManager.Save(CString)
1202-669 0.57 : 1202 CAPWorkSpaceMain.LoadScanModelMatrix(CString)
1202-1202 1.00 : 1202 CAPWorkSpaceMain.LoadScanModelMatrix(CString)
1202-1202 1.00 : 1202 CAPWorkSpaceMain.LoadScanModelMatrix(CString)
1202-1202 0.99 : 1202 CAPWorkSpaceMain.LoadScanModelMatrix(CString)
1202-1202 0.99 : 1202 CAPWorkSpaceMain.LoadScanModelMatrix(CString)
1204-1202 0.99 : 1204 CAPWorkSpaceMain.LoadBiteMatrix(CString)
1204-1202 0.99 : 1204 CAPWorkSpaceMain.LoadBiteMatrix(CStrin

In [218]:
import torch.nn.functional as F

result231 = embeded_tokens[idx]
result109 = result[0]
result59_2 = result2[0]

print(result231.shape)
print(result109.shape)
 
mean231 = torch.mean(result231)
mean109 = torch.mean(result109)

print("mean109 ", mean109)
print("mean231 ", mean231)

max231 = torch.max(result231)
max109 = torch.max(result109)

print("max231 ", max231)
print("max109 ", max109)

for i in range(109):
    scalar109 = torch.norm(result109[i])
    scalar231 = torch.norm(result231[i])
    cosine_similarity = F.cosine_similarity(result231[i], result109[i], dim=0)
    dot = F.cosine_similarity(result231[109+i], result231[109], dim=0)
    scal = torch.norm(result231[i+109])
    # print(f"{i-1}-{i} : {cosine_similarity}")
    print(f"{i} : {cosine_similarity:.2f} sim : 231(batch)/109 scalar {scalar231:.1f}/{scalar109:.1f} ::cycle:: {dot:.2f}/{scal:.2f}")

torch.Size([231, 768])
torch.Size([109, 768])
mean109  tensor(0.0216)
mean231  tensor(-0.0095)
max231  tensor(53.1944)
max109  tensor(53.1944)
0 : 1.00 sim : 231(batch)/109 scalar 39.5/39.5 ::cycle:: 1.00/39.54
1 : 1.00 sim : 231(batch)/109 scalar 39.5/39.5 ::cycle:: 1.00/39.54
2 : 1.00 sim : 231(batch)/109 scalar 39.5/39.5 ::cycle:: 1.00/39.54
3 : 1.00 sim : 231(batch)/109 scalar 44.6/44.6 ::cycle:: 1.00/39.54
4 : 1.00 sim : 231(batch)/109 scalar 48.6/48.6 ::cycle:: 1.00/39.54
5 : 1.00 sim : 231(batch)/109 scalar 44.7/44.7 ::cycle:: 1.00/39.54
6 : 1.00 sim : 231(batch)/109 scalar 51.2/51.2 ::cycle:: 1.00/39.54
7 : 1.00 sim : 231(batch)/109 scalar 46.2/46.2 ::cycle:: 1.00/39.54
8 : 1.00 sim : 231(batch)/109 scalar 42.5/42.5 ::cycle:: 1.00/39.54
9 : 1.00 sim : 231(batch)/109 scalar 47.4/47.4 ::cycle:: 1.00/39.54
10 : 1.00 sim : 231(batch)/109 scalar 45.2/45.2 ::cycle:: 1.00/39.54
11 : 1.00 sim : 231(batch)/109 scalar 47.1/47.1 ::cycle:: 1.00/39.54
12 : 1.00 sim : 231(batch)/109 scalar 4

In [45]:

for i in range(embeded_tokens)

OutOfMemoryError: CUDA out of memory. Tried to allocate 81.27 GiB (GPU 0; 12.00 GiB total capacity; 1.70 GiB already allocated; 8.23 GiB free; 2.73 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [39]:
del cuda_padded_tokens_ids
del cuda_attention_mask
del cuda_context_embeddings
torch.cuda.empty_cache()

In [44]:
import gc
gc.collect()
print(torch.cuda.memory_allocated())
print(torch.cuda.max_memory_allocated())

!nvidia-smi


1026445312
4551086080
Wed Aug 21 16:25:05 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.81                 Driver Version: 560.81         CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3060      WDDM  |   00000000:01:00.0  On |                  N/A |
|  0%   44C    P8             16W /  170W |    8508MiB /  12288MiB |      2%      Default |
|                                         |                        |                  N/A |
+-------------------------

In [43]:
print(torch.cuda.memory_allocated())
print(torch.cuda.max_memory_allocated())


1026445312
4551086080
